In [1]:
import time

# logic

- wait till nanonis starts
- read scanfile to get the current position of the nanonis
- start pulsed measurement
- wait until scanfile position changes
- stop measurement and save data

In [ ]:
refocus_time = 45 # seconds
pulsed_name = "rabi"

In [ ]:
pulsedmeasurement._pm.samplo_buttons[pulsed_name].click()
print("Loading {} to AWG".format(pulsed_name))

In [6]:
while True:
    if nanonis_raster_scanner.get_status()["nanonis_running"]:
        print("Nanonis is now running...")
        
        row, col = -1, -1
        
        while True:
            try:
                current_pos = nanonis_raster_scanner.get_pos(["current"])
                current_row, current_col = current_pos["row"], current_pos["col"]

                # Check if Nanonis position has changed
                if current_row != row or current_col != col:
                    # Fix for the case at (0, 0)
                    if current_row + current_col != 0:
                        # Stop pulsed measurement
                        pulsedmeasurement.measurement_run_stop_clicked(False)
                        print(f"[{row}, {col}] Stopping measurement '{pulsed_name}'")
                        
                        # Wait for stop
                        time.sleep(2)
                        
                        # Save pulsed measurement                                                
                        pulsedmeasurement.save_clicked()
                        print(f"[{row}, {col}] Saving data...")
                        time.sleep(2)
                    # End while loop after Nanonis finishes
                    if current_row == -1 and current_col == -1:
                        break
                        
                    # Set loop variables equal to position from Nanonis
                    row = current_row
                    col = current_col
                    
                    # Refocus tip with optimizer
                    confocalgui_ambient.refocus_clicked()
                    print(f"[{row}, {col}] Refocusing...")
                    time.sleep(refocus_time)

                    # Start pulsed measurement
                    pulsedmeasurement.measurement_run_stop_clicked(True)
                    pulsedmeasurement._mw.save_tag_LineEdit.setText(
                        "raster_{pulsed_name}_row{}_col{}".format(pulsed_name, row, col)
                    )
                    print(f"[{row}, {col}] Starting measurement '{pulsed_name}'...")
            except ValueError as exc:
                time.sleep(0.05)
                pass
        print("Nanonis has now stopped")
        break
    else:
        time.sleep(0.05)

In [ ]:
class NanonisRasterPulsed(GenericLogic):
    def __init__(self):
        self.pulsedmeasurement = pulsedmeasurement
        self.refocus_delay = 0
        self.refocus_on_each_point = False
    
    def start_measure(self):
        while True:
            if nanonis_raster_scanner.get_status()["nanonis_running"]:
                print("Nanonis is now running...")

                row, col = -1, -1

                while True:
                    try:
                        current_pos = nanonis_raster_scanner.get_pos(["current"])
                        current_row, current_col = current_pos["row"], current_pos["col"]

                        # Check if Nanonis position has changed
                        if current_row != row or current_col != col:
                            # Fix for the case at (0, 0)
                            if current_row + current_col != 0:
                                # Stop pulsed measurement
                                pulsedmeasurement.measurement_run_stop_clicked(False)
                                print(f"[{row}, {col}] Stopping measurement '{pulsed_name}'")

                                # Wait for stop
                                time.sleep(2)

                                # Save pulsed measurement                                                
                                pulsedmeasurement.save_clicked()
                                print(f"[{row}, {col}] Saving data...")
                                time.sleep(2)
                            # End while loop after Nanonis finishes
                            if current_row == -1 and current_col == -1:
                                break

                            # Set loop variables equal to position from Nanonis
                            row = current_row
                            col = current_col

                            # Refocus tip with optimizer
                            if self.refocus_on_each_point:
                                confocalgui_ambient.refocus_clicked()
                                print(f"[{row}, {col}] Refocusing...")
                                time.sleep(refocus_delay)

                            # Start pulsed measurement
                            pulsedmeasurement.measurement_run_stop_clicked(True)
                            pulsedmeasurement._mw.save_tag_LineEdit.setText(
                                "raster_{pulsed_name}_row{}_col{}".format(pulsed_name, row, col)
                            )
                            print(f"[{row}, {col}] Starting measurement '{pulsed_name}'...")
                    except ValueError as exc:
                        time.sleep(0.05)
                        pass
                print("Nanonis has now stopped")
                break
            else:
                time.sleep(0.05)

    def stop_measure(self):
        pass
    
    def set_up_measure(self, pulsed_name, refocus_on_each_point, refocus_delay=45):
        self.pulsedmeasurement._pm.samplo_buttons[pulsed_name].click()
        print("Loading {} to AWG".format(pulsed_name))
       

        self.refocus_on_each_point = refocus_on_each_point
        if self.refocus_on_each_point:
            self.refocus_delay = refocus_delay

In [ ]:
raster = NanonisRasterPulsed()
raster.set_up_measure(pulsed_name="rabi", refocus_on_each_point=True, refocus_delay=45)

In [ ]:
raster.start_measure()